In [1]:
# YOUR CODE HERE
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pltlab
import seaborn as sns
from sklearn.model_selection import train_test_split
import calendar
import geopandas as gpd
import censusdata
import warnings
import pipeline as pipe
warnings.filterwarnings('ignore')

%matplotlib inline

sns.set(rc={'figure.figsize':(11, 4)})

In [2]:
data = pd.read_pickle('data/final_dataset.pk1')

In [3]:
data.head()

,name,total_pop,median_income,state,county,county_name,state_name,prop_white,prop_black,prop_hisp,...,FIPS,covid_cases,Testing_Rate,gov_party,election_diff,Apr-19,Mar-19,Feb-20,Mar-20,days_closed
0,"Washington County, Mississippi",47086,30834.0,28,151,Washington,Mississippi,0.256913,0.721701,0.015482,...,28151.0,77,2291.374085,1,-0.364695,6.4,7.5,7.4,7,17.0
1,"Perry County, Mississippi",12028,39007.0,28,111,Perry,Mississippi,0.787745,0.196874,0.015048,...,28111.0,27,2291.374085,1,0.536635,5.7,6.2,6.8,6.3,17.0
2,"Choctaw County, Mississippi",8321,37203.0,28,019,Choctaw,Mississippi,0.676722,0.311982,0.003966,...,28019.0,13,2291.374085,1,0.386224,4.8,5.3,5.4,5,17.0
3,"Itawamba County, Mississippi",23480,40510.0,28,057,Itawamba,Mississippi,0.909114,0.071593,0.015332,...,28057.0,57,2291.374085,1,0.755161,4.1,4.4,4.7,4.7,17.0
4,"Carroll County, Mississippi",10129,43060.0,28,015,Carroll,Mississippi,0.643992,0.345839,0.002863,...,28015.0,38,2291.374085,1,0.383321,5.5,5.8,6.2,6,17.0


In [4]:
pipe.get_summary_stats(data)

total_pop  median_income   prop_white   prop_black    prop_hisp  \
count  2.826000e+03    2825.000000  2826.000000  2826.000000  2826.000000   
mean   1.117590e+05   51714.241416     0.827329     0.098748     0.091962   
std    3.414925e+05   13821.224638     0.164131     0.149708     0.135249   
min    4.180000e+02   20188.000000     0.093534     0.000000     0.000000   
25%    1.412200e+04   42491.000000     0.757614     0.008110     0.021500   
50%    3.013550e+04   49936.000000     0.889050     0.028182     0.041568   
75%    7.725500e+04   57886.000000     0.947572     0.117748     0.095686   
max    1.009805e+07  136268.000000     0.997743     0.874123     0.990688   

       log_med_income  prop_no_internet      prop_ba  prop_services  \
count     2825.000000       2826.000000  2826.000000    2825.000000   
mean        10.821023          0.225514     0.217787       0.083571   
std          0.251809          0.086132     0.096556       0.034479   
min          9.912844          0